# 埼玉県川越市　報道発表資料

埼玉県川越市が発表した新型コロナの患者情報のスクレイピングを行います。県内他の市町村から発表された情報は含まれていません。

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
import bs4
import re
import csv
import datetime
import requests
from collections import defaultdict

In [3]:
#export
urls = ['https://www.city.kawagoe.saitama.jp/kenkofukushi/byoki_iryo/kansensho/COVID19-p-C.html']

In [4]:
#export
def trim_div(s):
    return re.sub('<div>|<div .+?>|</div>', '', s)

In [5]:
#export
def get_src(url):
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    return bs4.BeautifulSoup(trim_div(response.text), 'html.parser')

In [6]:
#get_src(urls[0])

In [7]:
#export
def get_links(urls):
    domain = 'https://www.city.kawagoe.saitama.jp/'
    filter = '新型コロナウイルス.+患者.+発生'
    links = []
    for url in urls:
        src = get_src(url)
        links.extend(src.find_all(lambda tag: tag.name == 'a' and re.search(filter, tag.get_text()) != None))
    return [[domain + tag['href'], tag.get_text()] for tag in links]

In [8]:
links = get_links(urls)
links

[['https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0408.html',
  '新型コロナウイルスに関連した患者の発生について（4月8日発表分）'],
 ['https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0407.html',
  '新型コロナウイルスに関連した患者の発生について（4月7日発表分）'],
 ['https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0405.html',
  '新型コロナウイルスに関連した患者の発生について（4月5日発表分）'],
 ['https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0404.html',
  '新型コロナウイルスに関連した患者の発生について（4月4日発表分）'],
 ['https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0403.html',
  '新型コロナウイルスに関連した患者の発生について(4月3日発表分) ']]

In [9]:
def get_sample(urls, i):
    links = get_links(urls)
    return get_src(links[i][0])
#get_sample(urls, 0)

In [10]:
def show_contents(url):
    src = get_src(url)
    
    def get_text(tag):
        return tag.get_text().strip()
    
    h = src.body.find(lambda tag: tag.name == 'h3')
    for e in h.previous_sibling.next_siblings:
        if isinstance(e, bs4.element.Tag):
            if e.name == 'h2':
                break
            print(type(e), e.name, e.contents)

src = show_contents(links[0][0])
src

<class 'bs4.element.Tag'> h3 ['概要1(本市13例目)']
<class 'bs4.element.Tag'> ol [<li>年代：20歳代</li>, <li>性別：男性</li>, <li>職業：飲食業（東京都内勤務）</li>, <li>居住地：川越市</li>, <li>症状・経過<br/>4月1日：倦怠感あり。<br/>4月2日：出勤（20時～0時50分）。体温37.3度、咳症状あり。徐々に味覚嗅覚異常を感じる。<br/>4月4日：体温37.8度。市内Ａ医療機関を受診。<br/>4月6日：咳、全身倦怠感、味覚嗅覚異常が続いていたため、市内Ａ医療機関を受診。PCR検体採取。<br/>4月8日：PCR検査の結果、陽性判明。</li>, <li>渡航歴・患者との接触歴：なし</li>, <li>現在の症状：入院調整中。状態は安定している。</li>, <li>濃厚接触者：<br/>●家族4人<br/>　祖父：70歳代、症状なし・自宅待機中<br/>　祖母：70歳代、症状なし・自宅待機中<br/>　母：40歳代、症状なし・自宅待機中<br/>　同居人（男性）：40歳代、症状なし・自宅待機中<br/>●その他：現在調査中</li>, <li>その他<br/>・外出時は常にマスク着用。<br/>・発症日以降に利用した公共交通機関は4月4日のみで、副都心線東新宿から和光市乗り換えで東武東上線鶴ヶ島駅。<br/>・スポーツクラブ、大型商業施設等の利用なし。<br/></li>]
<class 'bs4.element.Tag'> h3 ['概要2(本市14例目)']
<class 'bs4.element.Tag'> ol [<li>年代：30歳代</li>, <li>性別：男性</li>, <li>職業：自営業</li>, <li>居住地：川越市</li>, <li>症状・経過<br/>3月30日：風邪症状あり。<br/>4月1日：出勤（マスク着用）。発熱はなし。咽頭の違和感あり、市内Ｂ医療機関受診。<br/>4月2日：体温38.5度。市内Ｂ医療機関を受診。<br/>4月4日：体温39.5度。<br/>4月6日：症状続いていたため、市内Ｂ医療機関を受診。<br/>4月7日：咳症状が悪化。市内Ｃ医療機関受診。PCR検体採取。<br/>4月8

In [11]:
#export
header = ['No','年代','性別','職業','居住地','症状・経過','渡航歴・患者との接触歴','現在の症状','濃厚接触者','その他','url']

In [12]:
#export
def parse(src, url):
    
    def get_text(tag):
        return tag.get_text().strip()
    
    def get_contents(src):
        h = src.body.find(lambda tag: tag.name == 'h3')
        return h.previous_sibling.next_siblings
    
    def parse_h3(e):
        no = re.sub("\\D", "", get_text(e))
        return no if len(no) > 0 else '0'
    
    def parse_ol(e):
        item = {}
        for li in e.find_all('li'):
            #print(type(li), li.name, li.get_text())
            if li.string:
                split = re.split('[：:]', get_text(li))
                if len(split) > 1:
                    item[split[0]] = split[1]
                else:
                    print(type(e), e.name, e.get_text())
            else:
                split = [c.string for c in li.contents if c.string] # <br/> を削除
                split[0] = re.sub('[：:]', '', split[0])
                item[split[0]] = split[1:]
        return item
        
    def dic2array(item):
        return [item[h] for h in header]

    patients = []
    item = defaultdict(lambda: '')
    for e in get_contents(src):
        if isinstance(e, bs4.element.Tag):
            if e.name == 'h2':
                break
            if e.name == 'h3':
                if len(item) > 0:
                    item[header[-1]] = url
                    patients.append(dic2array(item))
                    item = defaultdict(lambda: '')
                item[header[0]] = parse_h3(e)
            elif e.name == 'ol':
                item.update(parse_ol(e))
            else:
                print(type(e), e.name, e.get_text())
    if len(item) > 0:
        item['url'] = url
        patients.append(dic2array(item))
    return patients

In [13]:
#export
def get_patient(url):
    src = get_src(url)
    return parse(src, url)

In [14]:
get_patient(links[0][0])

[['113',
  '20歳代',
  '男性',
  '飲食業（東京都内勤務）',
  '川越市',
  ['4月1日：倦怠感あり。',
   '4月2日：出勤（20時～0時50分）。体温37.3度、咳症状あり。徐々に味覚嗅覚異常を感じる。',
   '4月4日：体温37.8度。市内Ａ医療機関を受診。',
   '4月6日：咳、全身倦怠感、味覚嗅覚異常が続いていたため、市内Ａ医療機関を受診。PCR検体採取。',
   '4月8日：PCR検査の結果、陽性判明。'],
  'なし',
  '入院調整中。状態は安定している。',
  ['●家族4人',
   '\u3000祖父：70歳代、症状なし・自宅待機中',
   '\u3000祖母：70歳代、症状なし・自宅待機中',
   '\u3000母：40歳代、症状なし・自宅待機中',
   '\u3000同居人（男性）：40歳代、症状なし・自宅待機中',
   '●その他：現在調査中'],
  ['・外出時は常にマスク着用。',
   '・発症日以降に利用した公共交通機関は4月4日のみで、副都心線東新宿から和光市乗り換えで東武東上線鶴ヶ島駅。',
   '・スポーツクラブ、大型商業施設等の利用なし。'],
  'https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0408.html'],
 ['214',
  '30歳代',
  '男性',
  '自営業',
  '川越市',
  ['3月30日：風邪症状あり。',
   '4月1日：出勤（マスク着用）。発熱はなし。咽頭の違和感あり、市内Ｂ医療機関受診。',
   '4月2日：体温38.5度。市内Ｂ医療機関を受診。',
   '4月4日：体温39.5度。',
   '4月6日：症状続いていたため、市内Ｂ医療機関を受診。',
   '4月7日：咳症状が悪化。市内Ｃ医療機関受診。PCR検体採取。',
   '4月8日：PCR検査の結果、陽性判明。'],
  'なし',
  '入院調整中。状態は安定している。',
  ['●家族3人',
   '\u3000妻：40歳代、症状なし・自宅待機',
   '\u3000長女：未就学、3月末頃より風邪症状あり・自宅待機中',

In [15]:
#export
def get_patients(urls):
    patients = []
    for link in get_links(urls):
        print(link)
        patients.extend(get_patient(link[0]))
    return patients

In [16]:
#export
def create_fname(base):
    now = datetime.datetime.now()
    return base + '_' + now.strftime('%Y%m%dT%H%M') + ".csv"

In [17]:
#export
def write_csv(patients, fname):
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(patients)

In [18]:
#export
def main():
    patients = sorted(get_patients(urls), key=lambda x: int(x[0]))
    patients.insert(0, header)
    fname = create_fname("data/11-city-kawagoe")
    write_csv(patients, fname)
    print("created:", fname)

In [19]:
#export
if __name__ == '__main__':
    main()

['https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0408.html', '新型コロナウイルスに関連した患者の発生について（4月8日発表分）']
['https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0407.html', '新型コロナウイルスに関連した患者の発生について（4月7日発表分）']
['https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0405.html', '新型コロナウイルスに関連した患者の発生について（4月5日発表分）']
['https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0404.html', '新型コロナウイルスに関連した患者の発生について（4月4日発表分）']
['https://www.city.kawagoe.saitama.jp//kenkofukushi/byoki_iryo/kansensho/COVID-19-p0403.html', '新型コロナウイルスに関連した患者の発生について(4月3日発表分) ']
created: data/11-city-kawagoe_20200409T2125.csv


In [21]:
# See: https://github.com/fastai/course-v3/blob/master/nbs/dl2/notebook2script.py
!python notebook2script.py 11-city-kawagoe_dev.ipynb

Converted 11-city-kawagoe_dev.ipynb to exp/nb_11-city-kawagoe.py
